In [1]:
# Import necessary packages
import math
import numpy as np
from sklearn.utils import shuffle
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

In [2]:
data = pd.read_csv("EEG Eye State.csv", skiprows = [i for i in range(19)], header = None)
values = data.values

In [3]:
Data = [i[:14] for i in values]

In [4]:
Data = np.transpose(Data)

In [5]:
Arranged = [i / max(abs(i)) for i in Data]

In [ ]:
np.savetxt('Normalized', Arranged, delimiter=',')

In [6]:
Normalized = np.transpose(Arranged)

In [7]:
print(len(Normalized))

14980


In [8]:
m = values.shape[0]
n = values.shape[1]
Data_0 = []
Data_1 = []
for i in range(m):
    if values[i][14] == 0:
        Data_0.append(Normalized[i,:14])
    elif values[i][14] == 1:
        Data_1.append(Normalized[i,:14])

print(Data_0[0])

[0.01399999 0.51369958 0.62338838 0.00645571 0.67192124 0.01264922
 0.00722333 0.63889952 0.01589382 0.63503617 0.617211   0.61127788
 0.03043767 0.00613754]


In [9]:
Data_Sequence = []
Data_Labels = []

length = 0
while length <= 4990:
    temp = []
    data_0 = Data_0[length:length+10]
    for i in range(14):
        attribute_data = [j[i] for j in data_0]
        temp.append(attribute_data)
    Data_Sequence.append(torch.tensor([temp]))
    Data_Labels.append(torch.tensor([0]))
    
    temp = []
    data_1 = Data_1[length:length+10]
    for i in range(14):
        attribute_data = [j[i] for j in data_1]
        temp.append(attribute_data)
    Data_Sequence.append(torch.tensor([temp]))
    Data_Labels.append(torch.tensor([1]))
    
    length += 10

print(Data_Sequence[0].shape)
# Test_Sequence = []
# Test_Labels = []
# while length <= 6000:
#     temp = []
#     data_0 = Data_0[length:length+100]
#     for i in range(14):
#         attribute_data = [j[i] for j in data_0]
#         temp.append(attribute_data)
#     Test_Sequence.append(torch.tensor([temp]))
#     Test_Labels.append(torch.tensor([0]))
    
#     temp = []
#     data_1 = Data_1[length:length+100]
#     for i in range(14):
#         attribute_data = [j[i] for j in data_1]
#         temp.append(attribute_data)
#     Test_Sequence.append(torch.tensor([temp]))
#     Test_Labels.append(torch.tensor([1]))
    
#     length += 100

torch.Size([1, 14, 10])


In [10]:
Data_Sequence, Data_Labels = shuffle(Data_Sequence, Data_Labels, random_state=0)

In [11]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(14, 200, 2)
        self.pool1 = nn.MaxPool1d(2,2)
        self.conv2 = nn.Conv1d(200, 200, 2)
        self.pool2 = nn.MaxPool1d(2,2)
        self.drop1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(200,2)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.drop1(x)
        x = x.view(-1, 200)
        x = F.relu(self.fc1(x))
        x = self.softmax(x)
        
        return x
        

model = Net()

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=math.exp(-4))

In [14]:
for epoch in range(10):
    train_loss = 0.0
    
    for i, data in enumerate(Data_Sequence):
        
        optimizer.zero_grad()
        
        output = model(data.float())
        target = Data_Labels[i]
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    print("train_loss: ", train_loss/len(Data_Sequence))
    
print("Finished Training")

train_loss:  0.8132616430521011
train_loss:  0.8132616430521011
train_loss:  0.8132616430521011
train_loss:  0.8132616430521011
train_loss:  0.8132616430521011
train_loss:  0.8132616430521011
train_loss:  0.8132616430521011
train_loss:  0.8132616430521011
train_loss:  0.8132616430521011
train_loss:  0.8132616430521011
Finished Training


In [ ]:
sample = Test_Sequence[10]
label = Test_Labels[10]
output = model(sample.float())
_, pred = torch.max(output,1)
print(label)
print(pred)